# Extended Kalman Filter & Smoother

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from glob import glob
from time import time
from scipy.stats import linregress
from lib import misc, utils, app
from lib.calib import project_points_fisheye

%load_ext autoreload
%autoreload 2

ROOT_DATA_DIR = os.path.join('..', 'data')

# Reconstruction Params
Define the params in the cell below. Thereafter, run all cells

In [ ]:
DATA_DIR = os.path.join(ROOT_DATA_DIR, '2019_03_09', 'lily', 'run')

start_frame = 70
end_frame = 170 # use -1 to reconstruct up to the last frame possible

# DLC p_cutoff - any points with likelihood < dlc_thresh are not trusted in optimization
dlc_thresh = 0.5 # change this only if the optimization result is unsatisfactory

# Optimization

In [ ]:
# ========= INIT VARS ========

t0 = time()

assert os.path.exists(DATA_DIR), f'Data directory not found: {DATA_DIR}'
OUT_DIR = os.path.join(DATA_DIR, 'ekf')
DLC_DIR = os.path.join(DATA_DIR, 'dlc')
assert os.path.exists(DLC_DIR), f'DLC directory not found: {DLC_DIR}'
os.makedirs(OUT_DIR, exist_ok=True)

app.start_logging(os.path.join(OUT_DIR, 'ekf.log'))

idx = misc.get_pose_params() # define the indices for the states
markers = misc.get_markers() # define DLC labels

n_markers = len(markers)
n_pose_params = len(idx)
n_states = 3*n_pose_params
vel_idx = n_states//3
acc_idx = n_states*2//3

derivs = {'d'+state: vel_idx+idx[state] for state in idx}
derivs.update({'d'+state: vel_idx+derivs[state] for state in derivs})
idx.update(derivs)

# load video info
res, fps, tot_frames, _ = app.get_vid_info(DATA_DIR) # path to original videos
assert end_frame <= tot_frames, f'end_frame must be less than or equal to {tot_frames}'
assert end_frame != 0, f'end_frame cannot be 0'
if end_frame < 0:
    end_frame = end_frame % tot_frames + 1 # cyclic
    
assert 0 < start_frame < tot_frames, f'start_frame must be strictly between 0 and {tot_frames}'
assert 0 <= dlc_thresh <= 1, 'dlc_thresh must be from 0 to 1'

with open(os.path.join(OUT_DIR, 'reconstruction_params.json'), 'w') as f:
    json.dump(dict(start_frame=start_frame, end_frame=end_frame, dlc_thresh=dlc_thresh), f)

# Load extrinsic params
k_arr, d_arr, r_arr, t_arr, cam_res, n_cams, scene_fpath = utils.find_scene_file(DATA_DIR)
assert res == cam_res
camera_params = [[K, D, R, T] for K, D, R, T in zip(k_arr, d_arr, r_arr, t_arr)]

# other vars
start_frame -= 1 # 0 based indexing
n_frames = end_frame-start_frame
sigma_bound = 3
max_pixel_err = cam_res[0] # used in measurement covariance R
sT = 1.0/fps # timestep

# ========= FUNCTION DEFINITINOS ========

def h_function(x: np.ndarray, k: np.ndarray, d: np.ndarray, r: np.ndarray, t: np.ndarray):
    """Returns a numpy array of the 2D marker pixel coordinates (shape Nx2) for a given state vector x and camera parameters k, d, r, t.
    """
    coords_3d = misc.get_3d_marker_coords(x)
    coords_2d = project_points_fisheye(coords_3d, k, d, r, t) # Project the 3D positions to 2D
    
    return coords_2d


def predict_next_state(x: np.ndarray, dt: np.float32):
    """Returns a numpy array of the predicted states for a given state vector x and time delta dt.
    """
    acc_prediction = x[acc_idx:]
    vel_prediction = x[vel_idx:acc_idx] + dt*acc_prediction
    pos_prediction = x[:vel_idx] + dt*vel_prediction + (0.5*dt**2)*acc_prediction
    
    return np.concatenate([pos_prediction, vel_prediction, acc_prediction]).astype(np.float32)


def numerical_jacobian(func, x: np.ndarray, *args):
    """Returns a numerically approximated jacobian of func with respect to x.
    Additional parameters will be passed to func using *args in the format: func(*x, *args)
    """
    n = len(x)
    eps = 1e-3
    
    fx = func(x, *args).flatten()
    xpeturb=x.copy()
    jac = np.empty((len(fx), n))
    for i in range(n):
        xpeturb[i] = xpeturb[i]+eps
        jac[:,i] = (func(xpeturb, *args).flatten() - fx)/eps
        xpeturb[i]=x[i]
        
    return jac


# ========= LOAD DLC DATA ========

# Load DLC 2D point files (.h5 outputs)
dlc_2d_point_files = sorted(glob(os.path.join(DLC_DIR, '*.h5')))
assert(len(dlc_2d_point_files) == n_cams), f'# of dlc .h5 files != # of cams in {n_cams}_cam_scene_sba.json'

# Load Measurement Data (pixels, likelihood)
points_2d_df = utils.load_dlc_points_as_df(dlc_2d_point_files, verbose=False)

# Restructure dataframe
points_df = points_2d_df.set_index(['frame', 'camera','marker'])
points_df = points_df.stack().unstack(level=1).unstack(level=1).unstack()

# Pixels array
pixels_df = points_df.loc[:, (range(n_cams), markers, ['x','y'])]
pixels_df = pixels_df.reindex(columns=pd.MultiIndex.from_product([range(n_cams), markers, ['x','y']]))
pixels_arr = pixels_df.to_numpy() #shape - (n_frames, n_cams * n_markers * 2)

# Likelihood array
likelihood_df = points_df.loc[:, (range(n_cams), markers, 'likelihood')]
likelihood_df = likelihood_df.reindex(columns=pd.MultiIndex.from_product([range(n_cams), markers, ['likelihood']]))
likelihood_arr = likelihood_df.to_numpy() #shape - (n_frames, n_cams * n_markers * 1)

# ========= INITIALIZE EKF MATRICES ========

# estimate initial points
states = np.zeros(n_states)

est_nose_pos, est_nose_vel = app.initialize_marker_3d(
    points_2d_df[points_2d_df['frame'].between(start_frame, end_frame-1)],
    'nose', k_arr, d_arr, r_arr, t_arr, plot=True
)

# INITIAL STATES
states[[idx['x_0'], idx['y_0'], idx['z_0']]] = est_nose_pos[0, :]   # head x, y, z in inertial
states[[idx['dx_0'], idx['dy_0'], idx['z_0']]] = est_nose_vel[0, :] # head x & y velocity in inertial
states[idx['psi_0']] = np.arctan2(*est_nose_vel[0, [1, 0]])         # head psi (yaw) in inertial

# INITIAL STATE COVARIANCE P - how much do we trust the initial states
# position
p_lin_pos = np.ones(3)*3**2                       # Know initial position within 4m
p_ang_pos = np.ones(n_pose_params-3)*(np.pi/4)**2 # Know initial angles within 60 degrees, heading may need to change
# p_lure_pos = p_lin_pos
# velocity
p_lin_vel = np.ones(3)*5**2                       # Know this within 2.5m/s and it's a uniform random variable
p_ang_vel = np.ones(n_pose_params-3)*3**2
# p_lure_vel = p_lin_vel
# acceleration
p_lin_acc = np.ones(3)*3**2
p_ang_acc = np.ones(n_pose_params-3)*3**2
p_ang_acc[10:] = 5**2
# p_lure_acc = p_lin_acc

P = np.diag(np.concatenate([p_lin_pos, p_ang_pos, #p_lure_pos,
                            p_lin_vel, p_ang_vel, #p_lure_vel,
                            p_lin_acc, p_ang_acc, #p_lure_acc
                           ]))

# PROCESS COVARIANCE Q - how 'noisy' the constant acceleration model is
qb_list = [
    5.0, 5.0, 5.0,    # head x, y, z in inertial
    10.0, 10.0, 10.0, # head phi, theta, psi in inertial
    5.0, 25.0, 5.0,   # neck phi, theta, psi
    50.0,             # front-torso theta
    5.0, 50.0, 25.0,  # back torso phi, theta, psi
    100.0, 30.0,      # tail base theta, psi
    140.0, 40.0,      # tail mid theta, psi
    350.0, 200.0,     # l_shoulder theta, l_front_knee theta
    350.0, 200.0,     # r_shoulder theta, r_front_knee theta
    450.0, 400.0,     # l_hip theta, l_back_knee theta
    450.0, 400.0,     # r_hip theta, r_back_knee theta
]
# qb_list += qb_list[0:3] # lure x, y, z in inertial - same as head

qb = (np.diag(qb_list)/2)**2
Q = np.block([
    [sT**4/4 * qb, sT**3/2 * qb, sT**2/2 * qb],
    [sT**3/2 * qb, sT**2 * qb, sT * qb],
    [sT**2/2 * qb, sT * qb, qb],
])

# MEASUREMENT COVARIANCE R
dlc_cov = 5**2

# State prediction function jacobian F - shape: (n_states, n_states)
rng = np.arange(n_states - vel_idx)
rng_acc = np.arange(n_states - acc_idx)
F = np.eye(n_states)
F[rng, rng+vel_idx] = sT
F[rng_acc, rng_acc+acc_idx] = sT**2/2

# Allocate space for storing EKF data
states_est_hist = np.zeros((n_frames, n_states))
states_pred_hist = states_est_hist.copy()
P_est_hist = np.zeros((n_frames, n_states, n_states))
P_pred_hist = P_est_hist.copy()

t1 = time()
print(f'\nInitialization took {t1-t0:.2f} seconds\n')

# ========= RUN EKF & SMOOTHER ========

t0 = time()

outliers_ignored = 0

for i in range(n_frames):
    print(f'Running frame {i+start_frame+1}\r', end='')
    
    # ========== PREDICTION ==========

    # Predict State
    states = predict_next_state(states, sT).flatten()
    states_pred_hist[i] = states

    # Projection of the state covariance
    P = F @ P @ F.T + Q
    P_pred_hist[i] = P
    
    # ============ UPDATE ============
    
    z_k = pixels_arr[i+start_frame]
    likelihood = likelihood_arr[i+start_frame]
    
    # Measurement
    H = np.zeros((n_cams*n_markers*2, n_states))
    h = np.zeros((n_cams*n_markers*2)) # same as H[:, 0].copy()
    for j in range(n_cams):
        # State measurement
        h[j*n_markers*2:(j+1)*n_markers*2] = h_function(states[:vel_idx], *camera_params[j]).flatten()
        # Jacobian - shape: (2*n_markers, n_states)
        H[j*n_markers*2:(j+1)*n_markers*2, 0:vel_idx] = numerical_jacobian(h_function, states[:vel_idx], *camera_params[j])
    
    # Measurement Covariance R
    bad_point_mask = np.repeat(likelihood<dlc_thresh, 2)
    dlc_cov_arr = dlc_cov*np.ones((n_cams*n_markers*2))
    dlc_cov_arr[bad_point_mask] = max_pixel_err # change this to be independent of cam res?
    R = np.diag(dlc_cov_arr**2)

    # Residual
    residual = z_k - h

    # Residual Covariance S
    S = (H @ P @ H.T) + R
    temp = sigma_bound*np.sqrt(np.diag(S)) # if measurement residual is worse than 3 sigma, set residual to 0 and rely on predicted state only
    for j in range(0, len(residual), 2):
        if np.abs(residual[j])>temp[j] or np.abs(residual[j+1])>temp[j+1]:
            residual[j:j+2] = 0
            outliers_ignored += 1

    # Kalman Gain
    K = P @ H.T @ np.linalg.inv(S)

    # Correction
    states = states + K @ residual
    states_est_hist[i] = states

    # Update State Covariance
    P = (np.eye(K.shape[0]) - K @ H) @ P
    P_est_hist[i] = P

print('EKF complete!')
print('Outliers ignored:', outliers_ignored)

# Run Kalman Smoother
smooth_states_est_hist = states_est_hist.copy()
smooth_P_est_hist = P_est_hist.copy()
for i in range(n_frames-2, 0, -1):
    A = P_est_hist[i] @ F.T @ np.linalg.inv(P_pred_hist[i+1])
    smooth_states_est_hist[i] = states_est_hist[i] + A @ (smooth_states_est_hist[i+1] - states_pred_hist[i+1])
    smooth_P_est_hist[i] = P_est_hist[i] + A @ (smooth_P_est_hist[i+1] - P_pred_hist[i+1]) @ A.T
    
print('\nKalman Smoother complete!\n')
t1 = time()
print(f'Optimization took {t1-t0:.2f} seconds\n')

app.stop_logging()

# Save EKF results

In [ ]:
states = dict(x=states_est_hist[:, :vel_idx],
              dx=states_est_hist[:, vel_idx:acc_idx],
              ddx=states_est_hist[:, acc_idx:],
              smoothed_x=smooth_states_est_hist[:, :vel_idx],
              smoothed_dx=smooth_states_est_hist[:, vel_idx:acc_idx],
              smoothed_ddx=smooth_states_est_hist[:, acc_idx:]
             )
app.save_ekf(states, OUT_DIR, scene_fpath, start_frame, dlc_thresh)

fig_fpath= os.path.join(OUT_DIR, 'ekf.svg')
app.plot_cheetah_states(states['x'], states['smoothed_x'], fig_fpath)

# Plot the cheetah!

In [ ]:
data_fpath = os.path.join(OUT_DIR, 'ekf.pickle')
app.plot_cheetah_reconstruction(data_fpath, reprojections=False, centered=True, dark_mode=True)